In [32]:
import pandas as pd
from pathlib import Path
import re
import numpy as np

In [12]:
example_path = Path('/gws/nopw/j04/iecdt/cheetah/2017_08_29/bottom/phantom/flick2')

In [59]:
class Sequence:
    def __init__(self, path: Path):
        self.path = path
        self.filtered_dir = self.path / "filtered_2D"
        self.samples = []
        # anything else we need to store?
    
    def generate_samples(self):

        # this will return a list(or other datastructure?) of Sample objects
        samples_list = []
        # first we need to load the dataframe for each camera file

        # get number of cameras properly
        num_cameras = 4 # (count number of camN files in the supplied directory)

        for file in self.filtered_dir.glob("cam*.h5"):
            match = re.search(r'cam(\d+)', file.name)
            if not match:
                print(f"Warning: Could not parse camera ID from {file.name}")
                continue

            cam_key = match.group(1)  # e.g. "3"
            print(f"Loading filtered detections from {file} ({cam_key})")

            df = pd.read_hdf(file, key="df_with_missing")

            print(df)
            # drop the uppermost level because we don't need it
            new_df = df.copy()
            new_df.columns = new_df.columns.droplevel(0)

            print("DROPPED COLUMNS")

            print(new_df)

            # go through the frames
            num_frames = len(new_df)

            bodyparts = new_df.columns.get_level_values("bodyparts").unique()
            num_bodyparts = len(bodyparts)

            # we know shape of tensor (C, J, 3)
            for i in range(num_frames):
                tensor = np.zeros((num_cameras, num_bodyparts, 3), dtype=np.float32)

                for body_idx, bodypart in enumerate(bodyparts):
                    u = new_df[(bodypart, 'x')].iloc[i]
                    v = new_df[(bodypart, 'y')].iloc[i]
                    c = new_df[(bodypart, 'likelihood')].iloc[i]

                    tensor[int(cam_key)-1, body_idx] = [u,v,c]

                    samples_list.append(tensor)
        

            




            


   
            # I want to read the dataframe and store as a tensor of shape (C, J, 3) where C is the number of cameras, J is the number of joints, and 3 is the (x,y,confidence) for each joint


        # one row of our dataframe, convert it to our "sample" object (whatever that may be)
#         for row in parsedfiles:
#             sample = Sample()
#             self.samples.append(sample)
    
#     def load_from_files(self):
#         # For each camera, load the undistorted 2D detections
#         self.filtered_detections = pd.DataFrame()  # Initialize as empty DataFrame

#         for file in self.filtered_dir.glob("cam*.h5"):
#             match = re.search(r'(cam\d+)', file.name)
#             if not match:
#                 print(f"Warning: Could not parse camera ID from {file.name}")
#                 continue

#             cam_key = match.group(1)  # e.g. "cam3"
#             print(f"Loading filtered detections from {file} ({cam_key})")

#             df = pd.read_hdf(file, key="df_with_missing")
#             df['camera'] = cam_key
#             df['frame'] = df['coords']

#             # we need to have 1 df per camera so num-cam dfs per file, then join them all into one df
#             self.filtered_detections = pd.concat([self.filtered_detections, df], ignore_index=True)





# class Sample:
#     def __init__(self, detections_2d, camera_proj, gt_3d):
#         self.detections_2d = detections_2d
#         self.camera_proj = camera_proj
#         self.gt_3d = gt_3d
    


                 
                 
        #           sequence: Path):
        # self.sequence = sequence
        # self.filtered_dir    = sequence / "filtered_2D"
        # self.gt_3d_path    = sequence / "gt_3D.pickle"

        # # these will be populated by load_from_files()
        # self.filtered_detections    = None
        # self.gt_3d                  = None






In [60]:
test = Sequence(example_path)

In [61]:
test.generate_samples()

Loading filtered detections from /gws/nopw/j04/iecdt/cheetah/2017_08_29/bottom/phantom/flick2/filtered_2D/cam3DLC_resnet152_CheetahOct14shuffle1_500000.h5 (3)
scorer    DLC_resnet152_CheetahOct14shuffle1_500000                         \
bodyparts                                     r_eye                          
coords                                            x           y likelihood   
0                                        466.780748  496.310020   0.843646   
1                                        476.662960  496.780764   0.867980   
2                                        467.291212  496.760568   0.816340   
3                                        467.653452  496.879998   0.833451   
4                                        476.905672  496.717315   0.845231   
..                                              ...         ...        ...   
316                                             NaN         NaN   0.031859   
317                                             NaN         N

IndexError: index 4 is out of bounds for axis 0 with size 4